In [ ]:
import numpy as np
import pandas as pd
import torch
import argparse
from torch.utils.data import DataLoader

from tqdm import tqdm
import os
import random
import sys
sys.path.append('/home/guest/DemandForecasting')

from model.dlinear import Model
from data_utils.load_data import load_and_preprocess_data

dataset, scaler = load_and_preprocess_data(
    data_path='/home/guest/DemandForecasting/data/imputed_data.csv',
    time_encoded=True,
    input_len=480,
    target_len=112
)

print(f'Total dataset size: {len(dataset)}')

# split train/val/test
num_train = int(len(dataset) * 0.99)
num_val = int(len(dataset) * (1 - 0.99) / 2)

train_dataset = torch.utils.data.Subset(dataset, list(range(0, num_train)))
test_dataset = torch.utils.data.Subset(dataset, list(range(num_train + num_val, len(dataset))))
del dataset

In [ ]:
from model.baseline_models import get_baseline_model, list_available_models

# Xem danh sách models
list_available_models()

# Naive baseline
model = get_baseline_model('naive', strategy='last')

# Moving Average
model = get_baseline_model('moving_average', window_length=7)

# Exponential Smoothing (cần fit với data)
model_class = get_baseline_model('exponential_smoothing', 
                                   trend='add', 
                                   seasonal='add', 
                                   seasonal_periods=12)
fitted = model_class(endog=train_data).fit()
forecast = fitted.forecast(steps=7)

# ARIMA (cần fit với data)
model_class = get_baseline_model('arima', order=(1, 1, 1))
fitted = model_class(endog=train_data).fit()
forecast = fitted.forecast(steps=7)

# LSTM (PyTorch)
model = get_baseline_model('lstm', input_size=1, hidden_size=64)
pred = model(x, n_ahead=7)